In [16]:
import os
import numpy as np
import pandas as pd

For els2002 institution selectivity classification; since F3PSSELECT variable (representing institutional selectivity classification) is based on the IPEDS 2010 Carnegie Classification: Undergraduate Profile, CCUGPROF variable, used the IPEDS 2010 standardized test score data for admissions for each college in the `MERGED2010_11_PP.csv` file.

Similarly for the hsls2009 institution selectivity classification, which used the IPEDS 2016 Carnegie Classification: Undergraduate Profile, so used the IPEDS 2016 standardized test score data for admissions for each college in the `MERGED2016_17_PP.csv` file.

In [17]:
scores_2012 = pd.read_csv('C:/Users/athen/Downloads/Capstone/data/raw_data/SAT_ACT_data/MERGED2010_11_PP.csv', delimiter=',', 
                          encoding='cp1252', low_memory=False)
scores_2016 = pd.read_csv('C:/Users/athen/Downloads/Capstone/data/raw_data/SAT_ACT_data/MERGED2016_17_PP.csv', delimiter=',', 
                          encoding='cp1252', low_memory=False)
most_recent_college = pd.read_csv('C:/Users/athen/Downloads/Capstone/data/raw_data/SAT_ACT_data/Most-Recent-Cohorts-Institution.csv', 
                                  delimiter=',', encoding='cp1252', low_memory=False)


college_2002 = pd.read_csv('C:/Users/athen/Downloads/Capstone/data/working_data/collegerecs/collegerec_els2002.csv')
college_2009 = pd.read_csv('C:/Users/athen/Downloads/Capstone/data/working_data/collegerecs/collegerec_hsls2009.csv')

Based on the columns in `MERGED2010_11_PP.csv` and `MERGED2016_17_PP.csv`, the columns related to SAT/ACT scores for each institution are represented by the following columns + definitions:

 - `SATVR25` - 25th percentile of SAT scores at the institution (critical reading)
 - `SATVR75` - 75th percentile of SAT scores at the institution (critical reading)
 - `SATMT25` - 25th percentile of SAT scores at the institution (math)
 - `SATMT75` - 75th percentile of SAT scores at the institution (math)
 - `SATWR25` - 25th percentile of SAT scores at the institution (writing)
 - `SATWR75` - 75th percentile of SAT scores at the institution (writing)
 - `SATVRMID` - Midpoint of SAT scores at the institution (critical reading)
 - `SATMTMID` - Midpoint of SAT scores at the institution (math)
 - `SATWRMID` - Midpoint of SAT scores at the institution (writing)
 - `ACTCM25` - 25th percentile of the ACT cumulative score
 - `ACTCM75` - 75th percentile of the ACT cumulative score
 - `ACTEN25` - 25th percentile of the ACT English score
 - `ACTEN75` - 75th percentile of the ACT English score
 - `ACTMT25` - 25th percentile of the ACT math score
 - `ACTMT75` - 75th percentile of the ACT math score
 - `ACTWR25` - 25th percentile of the ACT writing score
 - `ACTWR75` - 75th percentile of the ACT writing score
 - `ACTCMMID` - Midpoint of the ACT cumulative score
 - `ACTENMID` - Midpoint of the ACT English score
 - `ACTMTMID` - Midpoint of the ACT math score
 - `ACTWRMID` - Midpoint of the ACT writing score
 - `SAT_AVG` - Average SAT equivalent score of students admitted
 - `SAT_AVG_ALL` - Average SAT equivalent score of students admitted for all campuses rolled up to the 6-digit OPE ID


Given that the ACt has subsections for Science as well while all of the SAT subsections are accounted for in the above variables, and writing section is optional for the ACT, we determined that it would be best to keep only the 25th, 50th, and 75th percentiles for the SAT reading + math sections and the ACT cumulative values.

In [18]:
score_columns = ['UNITID','INSTNM','SATVR25','SATVR75','SATMT25','SATMT75',
                 'SATVRMID','SATMTMID','ACTCM25','ACTCM75','ACTCMMID']

scores_2012 = scores_2012[score_columns]
scores_2016 = scores_2016[score_columns]

In [19]:
scores_2012.shape

(7470, 11)

In [20]:
scores_2016.shape

(7238, 11)

In [21]:
# Dropping all rows where all relevant SAT/ACT test values are NA
scores_2012 = scores_2012.dropna(subset=['SATVR25','SATVR75','SATMT25','SATMT75',
                 'SATVRMID','SATMTMID',
                 'ACTCM25','ACTCM75','ACTCMMID'], how='all')

scores_2016 = scores_2016.dropna(subset=['SATVR25','SATVR75','SATMT25','SATMT75',
                 'SATVRMID','SATMTMID','ACTCM25','ACTCM75','ACTCMMID'], how='all')

In [22]:
# Merging previously created college_2002 and college_2009 datasets with the respective score data for that year's
# college application cycle
college_els2002 = pd.merge(college_2002, scores_2012, on='UNITID')
college_hsls2009 = pd.merge(college_2009, scores_2016, on='UNITID')

In [23]:
college_els2002.shape

(1435, 21)

In [24]:
college_hsls2009.shape

(1315, 21)

In [25]:
# Renaming els2002 colleges with most up-to-date name for the institution based on the university's id
college_els2002 = college_els2002.merge(most_recent_college[['UNITID', 'INSTNM']], on='UNITID', how='left')
college_els2002['INSTNM_x'] = college_els2002['INSTNM']
college_els2002 = college_els2002.dropna(subset=['INSTNM_x']).drop(['INSTNM_y', 'INSTNM'], axis=1)
college_els2002 = college_els2002.rename(columns={'INSTNM_x': 'INSTNM'})

In [26]:
college_els2002

,UNITID,INSTNM,selectivity,type,INSTSIZE,federalFinAid?,GROFFER,urbanicity,STABBR,HBCU,TRIBAL,SATVR25,SATVR75,SATMT25,SATMT75,SATVRMID,SATMTMID,ACTCM25,ACTCM75,ACTCMMID
0,100654,Alabama A & M University,3,1,2,1,1,1,AL,1,0,380.0,470.0,370.0,470.0,425.0,420.0,16.0,19.0,18.0
1,100663,University of Alabama at Birmingham,2,1,4,1,1,1,AL,0,0,490.0,680.0,510.0,690.0,585.0,600.0,21.0,27.0,24.0
2,100706,University of Alabama in Huntsville,2,1,3,1,1,1,AL,0,0,510.0,630.0,510.0,655.0,570.0,583.0,22.0,29.0,26.0
3,100724,Alabama State University,3,1,3,1,1,1,AL,1,0,360.0,440.0,320.0,420.0,400.0,370.0,13.0,17.0,15.0
4,100751,The University of Alabama,2,1,5,1,1,1,AL,0,0,490.0,620.0,500.0,620.0,555.0,560.0,22.0,29.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429,447847,Aviator College of Aeronautical Science and Te...,-1,3,1,1,2,4,FL,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,27.0,15.0
1431,449764,Visible Music College,-1,2,1,1,2,1,TN,0,0,360.0,560.0,330.0,600.0,460.0,465.0,19.0,25.0,22.0
1432,454184,The King's College,1,2,1,1,2,1,NY,0,0,570.0,690.0,510.0,620.0,630.0,565.0,24.0,30.0,27.0
1433,455770,Providence Christian College,2,2,1,1,2,1,CA,0,0,560.0,610.0,470.0,570.0,585.0,520.0,19.0,29.0,24.0


In [27]:
# Renaming hsls2009 colleges with most up-to-date name for the institution based on the university's id
college_hsls2009 = college_hsls2009.merge(most_recent_college[['UNITID', 'INSTNM']], on='UNITID', how='left')
college_hsls2009['INSTNM_x'] = college_hsls2009['INSTNM']
college_hsls2009 = college_hsls2009.dropna(subset=['INSTNM_x']).drop(['INSTNM_y', 'INSTNM'], axis=1)
college_hsls2009 = college_hsls2009.rename(columns={'INSTNM_x': 'INSTNM'})

In [28]:
college_hsls2009

,UNITID,INSTNM,selectivity,type,INSTSIZE,federalFinAid?,GROFFER,urbanicity,STABBR,HBCU,TRIBAL,SATVR25,SATVR75,SATMT25,SATMT75,SATVRMID,SATMTMID,ACTCM25,ACTCM75,ACTCMMID
0,100654,Alabama A & M University,3,1,3,1,1,1,AL,1,0,380.0,470.0,370.0,470.0,425.0,420.0,16.0,19.0,18.0
1,100663,University of Alabama at Birmingham,2,1,4,1,1,1,AL,0,0,480.0,640.0,490.0,660.0,560.0,575.0,21.0,28.0,25.0
2,100706,University of Alabama in Huntsville,2,1,3,1,1,1,AL,0,0,520.0,660.0,540.0,680.0,590.0,610.0,25.0,31.0,28.0
3,100724,Alabama State University,3,1,3,1,1,1,AL,1,0,370.0,460.0,360.0,460.0,415.0,410.0,15.0,19.0,17.0
4,100751,The University of Alabama,1,1,5,1,1,1,AL,0,0,490.0,610.0,490.0,620.0,550.0,555.0,23.0,31.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,486901,Milligan University,-1,2,2,1,1,2,TN,0,0,460.0,590.0,500.0,570.0,525.0,535.0,22.0,27.0,25.0
1311,487092,Emory University-Oxford College,-1,2,1,1,2,2,GA,0,0,610.0,700.0,620.0,760.0,655.0,690.0,28.0,32.0,30.0
1312,487524,Husson University,-1,2,2,1,1,1,ME,0,0,430.0,530.0,430.0,540.0,480.0,485.0,17.0,23.0,20.0
1313,488305,Elim Bible Institute and College,-1,2,1,1,2,3,NY,0,0,530.0,610.0,480.0,580.0,570.0,530.0,NaN,NaN,NaN


In [29]:
# Saves each resulting dataframe as separate datasets
np.savetxt('collegerec_els2002.csv', college_els2002, delimiter=',', header=','.join(list(college_els2002.columns)),  fmt='%s')

np.savetxt('collegerec_hsls2009.csv', college_hsls2009, delimiter=',', header=','.join(list(college_hsls2009.columns)), fmt='%s')